Example text

In [2]:
data = "example python code"
data

'example python code'

In [3]:
print(data)

example python code


Read data from csv

In [2]:
from pandas import read_csv
filename = 'dataset\\consumption_split\\CPI_ACPI_COI_RT_A-filtered-2021-03-21.csv'
dataset = read_csv(filename)

In [3]:
dataset.head()

,ref_area.label,indicator.label,source.label,classif1.label,time,obs_value,note_classif.label,note_indicator.label,note_source.label
0,Aruba,"National consumer price index (CPI), annual ra...",ABW - CPS - Consumer Price Survey,COICOP (Selected categories): General,1985,3.72,NaN,NaN,NaN
1,Aruba,"National consumer price index (CPI), annual ra...",ABW - CPS - Consumer Price Survey,COICOP (Selected categories): Food and non-alc...,1985,-0.20,NaN,NaN,NaN
2,Aruba,"National consumer price index (CPI), annual ra...",ABW - CPS - Consumer Price Survey,COICOP (Selected categories): Clothing and foo...,1985,0.79,NaN,NaN,NaN
3,Aruba,"National consumer price index (CPI), annual ra...",ABW - CPS - Consumer Price Survey,"COICOP (Selected categories): Electricity, gas...",1985,6.86,NaN,NaN,NaN
4,Aruba,"National consumer price index (CPI), annual ra...",ABW - CPS - Consumer Price Survey,COICOP (Selected categories): Actual and imput...,1985,2.84,NaN,NaN,NaN
